# ML2CPP

## Preparing the dataset

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

def populate_table(tablename, feature_names):
    iris = datasets.load_iris()
    X = iris.data  
    N = X.shape[0]
    y = iris.target.reshape(N,1)
    k = np.arange(N).reshape(N, 1)
    k_X_y = np.concatenate((k, X, y) , axis=1)
    lTable=pd.DataFrame(k_X_y)
    # print(lTable.head())
    lTable.columns = ['idx'] + feature_names + ['TGT'];
    lTable['TGT'] = lTable['TGT'].apply(int)
    lTable['idx'] = lTable['idx'].apply(int)
    lTable.to_csv(tablename , float_format='%.14g')



In [2]:
metadata = {"primary_key" : "KEY",
            "features" : ['sepal_length_cm', 'sepal_width_cm', 'petal_length_cm', 'petal_width_cm'],
            "targets" : ["TGT"],
            "table" : "iris"}

In [3]:
populate_table("/tmp/iris.csv" , metadata["features"])


In [4]:
df = pd.read_csv("/tmp/iris.csv")
df.sample(12, random_state=1960)

,Unnamed: 0,idx,sepal_length_cm,sepal_width_cm,petal_length_cm,petal_width_cm,TGT
114,114,114,5.8,2.8,5.1,2.4,2
74,74,74,6.4,2.9,4.3,1.3,1
9,9,9,4.9,3.1,1.5,0.1,0
88,88,88,5.6,3.0,4.1,1.3,1
25,25,25,5.0,3.0,1.6,0.2,0
5,5,5,5.4,3.9,1.7,0.4,0
48,48,48,5.3,3.7,1.5,0.2,0
117,117,117,7.7,3.8,6.7,2.2,2
83,83,83,6.0,2.7,5.1,1.6,1
105,105,105,7.6,3.0,6.6,2.1,2


## Training a Model

In [5]:


# train any scikit model on the iris dataset
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators = 32, max_depth=4, random_state=1960)
clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


<ipython-input-5-d4f6f82e69d7>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(df[metadata['features']].values, df[metadata['targets']].values)


ExtraTreesClassifier(max_depth=4, n_estimators=32, random_state=1960)

## Deploying the Model

In [6]:

def generate_cpp_for_model(model):
    import pickle, json, requests, base64
    b64_data = base64.b64encode(pickle.dumps(model)).decode('utf-8')
    # send the model th the web service
    json_data={"Name":"model_cpp_sample", 
               "PickleData":b64_data , 
               "SQLDialect":"CPP",
               "FeatureNames" : metadata['features']}
    r = requests.post("https://sklearn2sql.herokuapp.com/model", json=json_data)
    content = r.json()
    lCPP = content["model"]["SQLGenrationResult"][0]["SQL"]
    # print(lCPP);
    return lCPP


lCPPCode = generate_cpp_for_model(clf);


In [7]:
print(lCPPCode)

namespace  {

	namespace EXT_SubModel_0 {
	
		typedef std::vector<double> tNodeData;
		std::map<int, tNodeData> Decision_Tree_Node_data = {
				{ 4 ,  {1.0, 0.0, 0.0 }} ,
				{ 5 ,  {0.0, 1.0, 0.0 }} ,
				{ 7 ,  {0.0, 1.0, 0.0 }} ,
				{ 8 ,  {0.0, 0.0, 1.0 }} ,
				{ 10 ,  {1.0, 0.0, 0.0 }} ,
				{ 12 ,  {1.0, 0.0, 0.0 }} ,
				{ 13 ,  {0.0, 1.0, 0.0 }} ,
				{ 17 ,  {1.0, 0.0, 0.0 }} ,
				{ 18 ,  {0.0, 0.967741935483871, 0.03225806451612903 }} ,
				{ 20 ,  {0.0, 0.42424242424242425, 0.5757575757575758 }} ,
				{ 21 ,  {0.0, 0.0, 1.0 }} ,
				{ 22 ,  {1.0, 0.0, 0.0 }} 
		};
		
	
		int get_decision_tree_node_index(std::any Feature_0, std::any Feature_1, std::any Feature_2, std::any Feature_3) {
			int lNodeIndex = (Feature_0 <= 5.435769419801942) ? ( (Feature_1 <= 2.860070869597923) ? ( (Feature_3 <= 1.2930869178245756) ? ( (Feature_2 <= 2.8991271107602987) ? ( 4 ) : ( 5 ) ) : ( (Feature_3 <= 1.529976374366263) ? ( 7 ) : ( 8 ) ) ) : ( (Feature_3 <= 0.34153949482806456) ? ( 10 ) : ( 

In [8]:
    def write_text_to_file(iCPPCode, oCPPFile):          
        with open(oCPPFile, "w") as text_file:
            text_file.write(iCPPCode)

    def add_cpp_main_function(iCPPCode, iCSVFile):
        lCPPCode = "#include \"Generic.i\"\n\n"
        lCPPCode = lCPPCode + iCPPCode
        lCPPCode = lCPPCode + "\tint main() {\n"
        lCPPCode = lCPPCode + "\t\tscore_csv_file(\"" + iCSVFile +"\");\n"
        lCPPCode = lCPPCode + "\treturn 0;\n}\n"
        return lCPPCode

    def compile_cpp_code_as_executable(iName):
        import subprocess
        lCommand = ["g++", "-Wall", "-Wno-unused-function", "-std=c++17" , "-g" ,  "-o", iName + ".exe",  iName + ".cpp"]
        print("EXECUTING" , "'" + " ".join(lCommand) + "'")
        result = subprocess.check_output(lCommand)
        # print(result)

    def execute_cpp_model(iName, iCSVFile):
        import subprocess
        result2 = subprocess.check_output([iName + ".exe",  iCSVFile])
        result2 = result2.decode()
        print(result2[:1000])
        print(result2[-1000:])
        return result2
        
    def execute_cpp_code(iCPPCode, iCSVFile):
        lName = "/tmp/sklearn2sql_cpp_" + str(id(clf));
        lCPPCode = add_cpp_main_function(iCPPCode, iCSVFile)
        write_text_to_file(lCPPCode, lName + ".cpp")
        compile_cpp_code_as_executable(lName)
        result = execute_cpp_model(lName, iCSVFile)
        write_text_to_file(str(result), lName + ".out")
        return lName + ".out"


In [9]:
populate_table("/tmp/iris2.csv" , ["Feature_0", "Feature_1", "Feature_2", "Feature_3"])
lCPPOutput = execute_cpp_code(lCPPCode , "/tmp/iris2.csv")
cpp_output = pd.read_csv(lCPPOutput)

EXECUTING 'g++ -Wall -Wno-unused-function -std=c++17 -g -o /tmp/sklearn2sql_cpp_140363236477584.exe /tmp/sklearn2sql_cpp_140363236477584.cpp'
idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
0,,,,0.99322916666667,0.00590277777778,0.00086805555556,-0.00679385942141,-5.13233222907378,-7.04925484125584,0,0.99322916666667
1,,,,0.99157196969697,0.00842803030303,0.00000000000000,-0.00846374697279,-4.77619218752811,-inf,0,0.99157196969697
2,,,,0.99157196969697,0.00842803030303,0.00000000000000,-0.00846374697279,-4.77619218752811,-inf,0,0.99157196969697
3,,,,0.99157196969697,0.00842803030303,0.00000000000000,-0.00846374697279,-4.77619218752811,-inf,0,0.99157196969697
4,,,,0.99322916666667,0.00590277777778,0.00086805555556,-0.00679385942141,-5.13233222907378,-7.04925484125584,0,0.99322916666667
5,,,,0.97135416666667,0.02256944444444,0.00607638888889,-0.02906413294391,-3.79115830323435,-5.10334469220052,0,0.97135416666667
6,,,,0.99322916

In [10]:
cpp_output.sample(12, random_state=1960)

,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision,DecisionProba
114,114,NaN,NaN,NaN,0.007610,0.139152,0.853238,-4.878257,-1.972189,-0.158717,2,0.853238
74,74,NaN,NaN,NaN,0.003779,0.867923,0.128297,-5.578198,-0.141652,-2.053405,1,0.867923
9,9,NaN,NaN,NaN,0.991572,0.008428,0.000000,-0.008464,-4.776192,-inf,0,0.991572
88,88,NaN,NaN,NaN,0.044337,0.834264,0.121398,-3.115932,-0.181205,-2.108678,1,0.834264
25,25,NaN,NaN,NaN,0.962338,0.028085,0.009577,-0.038389,-3.572509,-4.648431,0,0.962338
5,5,NaN,NaN,NaN,0.971354,0.022569,0.006076,-0.029064,-3.791158,-5.103345,0,0.971354
48,48,NaN,NaN,NaN,0.993229,0.005903,0.000868,-0.006794,-5.132332,-7.049255,0,0.993229
117,117,NaN,NaN,NaN,0.000000,0.020891,0.979109,-inf,-3.868414,-0.021113,2,0.979109
83,83,NaN,NaN,NaN,0.002402,0.570999,0.426599,-6.031482,-0.560368,-0.851910,1,0.570999
105,105,NaN,NaN,NaN,0.001276,0.052055,0.946670,-6.664409,-2.955458,-0.054805,2,0.946670


In [11]:
skl_outputs = pd.DataFrame()
X = df[metadata['features']].values
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['idx']);

skl_output_score = pd.DataFrame(columns=['Score_0', 'Score_1', 'Score_2']);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_0', 'Proba_1', 'Proba_2'])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_0', 'LogProba_1', 'LogProba_2'])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)

/home/antoine/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:721: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


,idx,Score_0,Score_1,Score_2,Proba_0,Proba_1,Proba_2,LogProba_0,LogProba_1,LogProba_2,Decision
114,114,NaN,NaN,NaN,0.007610,0.139152,0.853238,-4.878257,-1.972189,-0.158717,2
74,74,NaN,NaN,NaN,0.003779,0.867923,0.128297,-5.578198,-0.141652,-2.053405,1
9,9,NaN,NaN,NaN,0.991572,0.008428,0.000000,-0.008464,-4.776192,-inf,0
88,88,NaN,NaN,NaN,0.044337,0.834264,0.121398,-3.115932,-0.181205,-2.108678,1
25,25,NaN,NaN,NaN,0.962338,0.028085,0.009577,-0.038389,-3.572509,-4.648431,0
5,5,NaN,NaN,NaN,0.971354,0.022569,0.006076,-0.029064,-3.791158,-5.103345,0
48,48,NaN,NaN,NaN,0.993229,0.005903,0.000868,-0.006794,-5.132332,-7.049255,0
117,117,NaN,NaN,NaN,0.000000,0.020891,0.979109,-inf,-3.868414,-0.021113,2
83,83,NaN,NaN,NaN,0.002402,0.570999,0.426599,-6.031482,-0.560368,-0.851910,1
105,105,NaN,NaN,NaN,0.001276,0.052055,0.946670,-6.664409,-2.955458,-0.054805,2


In [12]:
cpp_skl_join = skl_output.join(cpp_output , how='left', on='idx', lsuffix='_skl', rsuffix='_cpp')

In [13]:
cpp_skl_join.sample(12, random_state=1960)

,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
114,114,NaN,NaN,NaN,0.007610,0.139152,0.853238,-4.878257,-1.972189,-0.158717,...,NaN,NaN,0.007610,0.139152,0.853238,-4.878257,-1.972189,-0.158717,2,0.853238
74,74,NaN,NaN,NaN,0.003779,0.867923,0.128297,-5.578198,-0.141652,-2.053405,...,NaN,NaN,0.003779,0.867923,0.128297,-5.578198,-0.141652,-2.053405,1,0.867923
9,9,NaN,NaN,NaN,0.991572,0.008428,0.000000,-0.008464,-4.776192,-inf,...,NaN,NaN,0.991572,0.008428,0.000000,-0.008464,-4.776192,-inf,0,0.991572
88,88,NaN,NaN,NaN,0.044337,0.834264,0.121398,-3.115932,-0.181205,-2.108678,...,NaN,NaN,0.044337,0.834264,0.121398,-3.115932,-0.181205,-2.108678,1,0.834264
25,25,NaN,NaN,NaN,0.962338,0.028085,0.009577,-0.038389,-3.572509,-4.648431,...,NaN,NaN,0.962338,0.028085,0.009577,-0.038389,-3.572509,-4.648431,0,0.962338
5,5,NaN,NaN,NaN,0.971354,0.022569,0.006076,-0.029064,-3.791158,-5.103345,...,NaN,NaN,0.971354,0.022569,0.006076,-0.029064,-3.791158,-5.103345,0,0.971354
48,48,NaN,NaN,NaN,0.993229,0.005903,0.000868,-0.006794,-5.132332,-7.049255,...,NaN,NaN,0.993229,0.005903,0.000868,-0.006794,-5.132332,-7.049255,0,0.993229
117,117,NaN,NaN,NaN,0.000000,0.020891,0.979109,-inf,-3.868414,-0.021113,...,NaN,NaN,0.000000,0.020891,0.979109,-inf,-3.868414,-0.021113,2,0.979109
83,83,NaN,NaN,NaN,0.002402,0.570999,0.426599,-6.031482,-0.560368,-0.851910,...,NaN,NaN,0.002402,0.570999,0.426599,-6.031482,-0.560368,-0.851910,1,0.570999
105,105,NaN,NaN,NaN,0.001276,0.052055,0.946670,-6.664409,-2.955458,-0.054805,...,NaN,NaN,0.001276,0.052055,0.946670,-6.664409,-2.955458,-0.054805,2,0.946670


In [14]:
condition = (cpp_skl_join.Decision_cpp != cpp_skl_join.Decision_skl)
cpp_skl_join[condition]


,idx_skl,Score_0_skl,Score_1_skl,Score_2_skl,Proba_0_skl,Proba_1_skl,Proba_2_skl,LogProba_0_skl,LogProba_1_skl,LogProba_2_skl,...,Score_1_cpp,Score_2_cpp,Proba_0_cpp,Proba_1_cpp,Proba_2_cpp,LogProba_0_cpp,LogProba_1_cpp,LogProba_2_cpp,Decision_cpp,DecisionProba
